In [1]:
import numpy as np
import argparse
import cv2
import os

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=False, help="path to input image", default='MobileNet_SSD/images/example_01.jpg')
# ap.add_argument("-p", "--prototxt", required=False, default='MobileNet_SSD/MobileNetSSD_deploy.prototxt.txt', help="path to Caffe 'deploy' prototxt file")
# ap.add_argument("-m", "--model", required=False, default='MobileNet_SSD/MobileNetSSD_deploy.caffemodel', help="path to Caffe pre-trained model")
# ap.add_argument("-c", "--confidence", type=float, default=0.2, help="minimum probability to filter weak detections")
# args = vars(ap.parse_args())
args = {
    'model': 'MobileNet_SSD/MobileNetSSD_deploy.caffemodel',
    'prototxt': 'MobileNet_SSD/MobileNetSSD_deploy.prototxt.txt',
    'image': 'MobileNet_SSD/images/example_01.jpg',
    'folder': 'MobileNet_SSD/images/',
    'confidence': 0.2
}

In [2]:
import time

In [3]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

In [4]:
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [5]:
image = cv2.imread(args["image"])
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

In [6]:
# cv2.imshow('image', image)

In [7]:
print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

[INFO] computing object detections...


In [8]:
print(detections)
detections.shape
detections[0, 0, 0, 2]

[[[[0.         7.         0.9995229  0.5379815  0.5264416  0.88357186
    0.8250202 ]
   [0.         7.         0.9561934  0.10046941 0.4999609  0.35983586
    0.7624899 ]]]]


0.9995229

In [9]:
for i in np.arange(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the
    # prediction
    confidence = detections[0, 0, i, 2]
    print(confidence)
    print(type(confidence))
    # filter out weak detections by ensuring the `confidence` is
    # greater than the minimum confidence
    if confidence > args["confidence"]:
        # extract the index of the class label from the `detections`,
        # then compute the (x, y)-coordinates of the bounding box for
        # the object
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        # display the prediction
        label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
        print("[INFO] {}".format(label))
        cv2.rectangle(image, (startX, startY), (endX, endY),
            COLORS[idx], 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(image, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
        print(cv2.imwrite(args['image'].split('/')[0] + '/outputs/' + args['image'].split('/')[-1], image))

0.9995229
<class 'numpy.float32'>
[INFO] car: 99.95%
True
0.9561934
<class 'numpy.float32'>
[INFO] car: 95.62%
True


In [13]:
# cv2.imshow("Output", image)
# MobileNet_SSD/images